In [43]:
import sklearn
import pandas as pd

In [44]:
data = pd.read_csv('dataset.csv')
data.head()

,question,label
0,Är det lämpligt att gå med i ett välrenommerat...,0
1,Vilka städer nämner president Zelenskyj som nu...,1
2,Hur kan jag få en tjej att verkligen bli kär i...,0
3,"Vilka är möjliga orsaker till illamående, hals...",0
4,Vad hävdar Ryska försvarsdepartementet att de ...,1


In [45]:
# Divide the data into train and test
training_data, test_data = sklearn.model_selection.train_test_split(data, test_size = 0.15, random_state = 1)
print("training data: ", training_data.shape)
print("testing data: ", test_data.shape)

training data:  (6800, 2)
testing data:  (1200, 2)


In [46]:
pipeline = sklearn.pipeline.Pipeline([
    ('vectorizer', sklearn.feature_extraction.text.CountVectorizer()),
    ('classifier', sklearn.linear_model.SGDClassifier(random_state=1, max_iter=3000))
])

parameters = {
    'vectorizer': [sklearn.feature_extraction.text.CountVectorizer(), sklearn.feature_extraction.text.TfidfVectorizer()],
    'vectorizer__binary': [False, True],
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'classifier__alpha': [0.05, 0.08, 0.1, 0.3, 0,5],
    'classifier__penalty': ['l2', 'l1', 'elasticnet'],
    'classifier__loss': ['hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron'],
}

grid_search = sklearn.model_selection.GridSearchCV(pipeline, parameters, cv = 5, n_jobs = -1, verbose = 1)
grid_search.fit(training_data['question'], training_data['label'])

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_mo

Best score: 0.953
Best parameters set:
	classifier__alpha: 0.08
	classifier__loss: 'perceptron'
	classifier__penalty: 'l2'
	vectorizer: TfidfVectorizer(binary=True, ngram_range=(1, 2))
	vectorizer__binary: True
	vectorizer__ngram_range: (1, 2)


/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
600 fits failed out of a total of 3600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
600 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py", line 892, in fit
    self._more_validate_params()
  File 

In [49]:
pipelineBestParams = sklearn.pipeline.Pipeline([
    ('vectorizer', sklearn.feature_extraction.text.TfidfVectorizer(binary=True, ngram_range=(1, 2))),
    ('classifier', sklearn.linear_model.SGDClassifier(random_state=1, max_iter=3000, alpha=0.08, loss = "perceptron", penalty = "l2"))
])

pipelineBestParams.fit(training_data['question'], training_data['label'])
predictions = pipelineBestParams.predict(test_data['question'])

report = sklearn.metrics.classification_report(test_data['label'], predictions)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.96      0.96       627
           1       0.96      0.96      0.96       573

    accuracy                           0.96      1200
   macro avg       0.96      0.96      0.96      1200
weighted avg       0.96      0.96      0.96      1200

